# Anonymise data

### Removing names/email addresses from the survey data 
#### Save the dataframes to pickle files

In [2]:
import json
import pickle
from pandas.io.json import json_normalize
import os
import logging
import logging.handlers
import pandas as pd
import numpy as np
import glob
from datetime import datetime,timedelta
from collections import Counter
import nltk 
from functools import reduce
import re
from scipy.stats import pointbiserialr
import matplotlib
import matplotlib.pyplot as plt
from matplotlib_venn import venn3
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
matplotlib.style.use('ggplot')
import seaborn as seabornInstance 
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

In [3]:
def dt_parse(t):
    ''' 
    Function that parses datetime so it can be used to find duration of conversation
    INPUT: a string t representing a message timestamp
    OUTPUT: a string ret representing the message timestamp in the format %Y-%m-%dT%H:%M
    '''
    ret = datetime.strptime(t[0:16],'%Y-%m-%dT%H:%M')
    return ret 

In [4]:
def create_utterance_df(rootDir):
    '''
    Function that traverses subfolders of a root directory 
    where each subfolder corresponds to a conversation with one of the chatbots.
    Each subfolder contains JSON files, each corresponding to a single user message. 
    '''
    
    jsdata = []
    df = pd.DataFrame(pd.np.empty((0, 7)))
    df.columns = ['Username','Duration', 'Chatbot', 'NumberOfTurns', 'Utterance','Word_Count', "Avg_Utt_Len"]

    for subdir, dirs, files in os.walk(rootDir): # each conversation has its own folder of json files
        for dir_ in dirs: # for each conversation
            message_timestamps = []
            user_utterances = []
            char_counts = []
            numberOfTurns = 0
            word_count = 0
            for file in glob.glob(rootDir+dir_+'/'+'*.json'):
                with open(file) as f:
                    data = json.load(f)
                    numberOfTurns += 1
                    message_timestamps.append(dt_parse(data['timestamp']))
                    user_utterances.append(data['text'])
                    for word in (data['text']).split():
                        word_count += 1
                    char_counts.append(len(data['text']) - data['text'].count(' '))
            avg_char_count = np.mean(char_counts)       


            #add post interaction survey (Nasoto)

            #Duration
            maxTime = max(dt for dt in message_timestamps)
            minTime = min(dt for dt in message_timestamps)
            duration =  maxTime - minTime

            #get the information from nasoto inserted
            jsdata.append({'Username': data['from']['name'],
                           'Duration': duration,'Chatbot': data['recipient']['name'], 
                           'NumberOfTurns' : numberOfTurns, 'Utterance': user_utterances, 
                            'Word_Count': word_count, 'Avg_Utt_Len': avg_char_count})



    df =  pd.DataFrame.from_records(jsdata)
    df[:]['Duration'] / np.timedelta64(1, 'm')
    return df  

## Interaction DataFrames

In [5]:
df_nasoto = create_utterance_df('webchat-nasoto-userstudy/')
df_makoto = create_utterance_df('webchat-makoto-userstudy/')

/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



# Survey DataFrames 

In [6]:
PostMakotoDir = 'surveys-userstudy/Post-Interaction Survey (MAKOTO).csv'
df_PostMakoto = pd.read_csv(PostMakotoDir, error_bad_lines=False)
df_PostMakoto['Username'] =  df_PostMakoto['Username'].str.replace(r'@ucdconnect.ie', '')

In [7]:
PostNasotoDir = 'surveys-userstudy/Post-Interaction Survey (NASOTO).csv'
df_PostNasoto = pd.read_csv(PostNasotoDir, error_bad_lines=False)
df_PostNasoto['Username'] =  df_PostNasoto['Username'].str.replace(r'@ucdconnect.ie', '')

## Demographic Survey

In [8]:
PreInteractionPath = 'surveys-userstudy/Pre-Interaction Survey.csv'
df_dem = pd.read_csv(PreInteractionPath,  error_bad_lines=False)
df_dem['Username'] =  df_dem['Username'].str.replace(r'@ucdconnect.ie', '')

In [12]:
Survey4Dir = 'surveys-userstudy/Final Survey .csv'
df_survey4 = pd.read_csv(Survey4Dir)
df_survey4['Username'] =  df_survey4['Username'].str.replace(r'@ucdconnect.ie', '')
df_survey4

,Timestamp,Username,Did you notice any differences in your interactions with one chatbot over the other? If so what were they.,Which chatbot did you prefer interacting with?,Explain why you preferred interacting with this chatbot?
0,2020/04/11 12:58:39 pm CET,george.ridgway,"Yes, I felt like the Nasoto allowed for a much...",Nasoto,The conversation I had with Nasoto felt two-si...
1,2020/04/11 1:30:37 pm CET,oisin.quinn,Nasoto felt more formal. It felt like I was do...,Makoto,They were just more fun and relaxed and it was...
2,2020/04/11 11:39:16 pm CET,mohamed.jama,yes. I found the first bot to be more pleasent...,Makoto,it is not rude and the questions it asked allo...
3,2020/04/12 2:12:37 pm CET,nikolaj.jasenko,"Yeah, first one was more friendly",Makoto,More interesting to interact with due to a bri...
4,2020/04/15 5:01:59 pm CET,kiowa.daly,"Yes, the first bot seems to disregard your res...",Makoto,"Makoto is more cheery, nasoto seems to not res..."
5,2020/04/15 5:14:04 pm CET,conor.beenham,"Yes. Makoto was friendlier, and seemed more 'i...",Makoto,"resulted in a better, more interesting convers..."
6,2020/04/15 6:01:50 pm CET,zachary.oconnor,"They asked the same kinds of questions, but Ma...",Makoto,Makoto seemed nicer and more excited to talk t...
7,2020/04/15 6:46:40 pm CET,orla.keating,"Yes I talked to one longer than the other, alt...",Nasoto,It was engaging more in the conversation
8,2020/04/15 10:02:56 pm CET,daire.murphy.2,The second bots personality was much nicer to ...,Makoto,It was sounder to chat to. Better personality
9,2020/04/16 2:50:45 pm CET,jeremiah.wangaruro,Makoto understood more of what i said,Makoto,Better understanding


# Create Dataframes

In [13]:
df_allSurveys =  [df_dem, df_PostMakoto, df_PostNasoto, df_survey4, df_nasoto, df_makoto]


df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Username'],
                                            how='outer'), df_allSurveys)
df_merged = df_merged[:-1]


In [14]:
df_merged['Chatbot_binary'] = pd.np.where(df_merged['Which chatbot did you prefer interacting with?'].str.contains("Makoto"), 0, 1)


/Users/sineadfarrell/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



# Anonymise username

In [15]:
df_merged.drop(columns=["Username"], inplace=True)
df_merged.to_pickle("./df_merged.pkl")

In [16]:
df_dem.drop(columns=["Username"], inplace= True)
df_dem.to_pickle("./df_dem.pkl")

In [17]:
frames = [df_nasoto, df_makoto]
df_chatbotdata =  pd.concat(frames)
df_chatbotdata.drop(columns=["Username"], inplace= True)
df_chatbotdata.to_pickle("./df_chatbotdata.pkl")